In [12]:
import pandas as pd
import numpy as np
import time
import re as re
from selenium import webdriver
import translators as ts
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
def get_data_velux(url):
    
    
    """
    This function will collect the grant information provided from the Velux foundation grant database with the url: https://veluxfonden.dk/da/det-har-vi-stoettet?calcbrowser.view_name.value=bars_horizontal_area&calcbrowser.table-sort=year"
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    title_year = []
    project_category_l = []
    receiver_l = []
    amount_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url ("https://veluxfonden.dk/da/det-har-vi-stoettet?calcbrowser.view_name.value=bars_horizontal_area&calcbrowser.table-sort=year")
    return: Dataframe with text extracted
    """

    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    
    title_year = []
    project_category_l = []
    receiver_l = []
    amount_l = []
    
    
    time.sleep(5)
    
    #click on table button
    ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[1]/div[2]/a[2]")).perform()  #move to place
    time.sleep(5)
    driver.find_element(By.XPATH, "/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[1]/div[2]/a[2]").click()
    #element.click()
    
    try:
        for page_num in range(1, 30):
            for i in range(1, 51):
                #move to your target
                ActionChains(driver).move_to_element(driver.find_element(By.XPATH, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[1]")).perform()
                title = driver.find_element(By.XPATH, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[1]").text
                project_cat = driver.find_element(By.XPATH, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[4]").text
                receiver = driver.find_element(By.XPATH, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[3]").text
                amount = driver.find_element(By.XPATH, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[2]/span[2]").text

                title_year.append(title)
                project_category_l.append(project_cat)
                receiver_l.append(receiver)
                amount_l.append(amount)
                
            ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[3]/a[2]/span")).perform()  #move to click button
            time.sleep(5)
            driver.find_element(By.XPATH, "/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[3]/a[2]").click()
                
    except NoSuchElementException: #avoid programme crashing
        print("ended loop")
    
    df_final = pd.DataFrame(list(zip(title_year, project_category_l, receiver_l, amount_l)), columns=["Title", "Project Category", "Receiver Name", 'Grant size (DKK)'])


    return df_final

big_list_velux = get_data_velux("https://veluxfonden.dk/da/det-har-vi-stoettet?calcbrowser.view_name.value=bars_horizontal_area&calcbrowser.table-sort=year")
pd.set_option('display.max_columns', None)
big_list_velux.to_csv('velux_data.csv', index=True)

Loop ended
